### Попробуем решить задачу бинарной классификации, определяя по погодным признакам одну из столиц - Москву или Санкт-Петербург.

Для всех моделей метрика - ROC-AUC.

Хорошо подходит для случаев, когда оба класса, во-первых, сбалансированы, во-вторых - одинаково важны.

А это как раз наш случай. 

In [2]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.stats as sts
#import seaborn as sns

In [140]:
import warnings
warnings.simplefilter('ignore')

In [82]:
# техническое извлечение таблиц
df_spb = pd.read_csv('final_spb', sep=',')
df_moscow = pd.read_csv('final_moscow', sep=',')

In [83]:
# обработка значений для машинного обучения
df_spb.drop(columns='Дата', inplace=True) # конкретные даты нам уже не понадобятся
df_moscow.drop(columns='Дата', inplace=True)
df_spb['Город'] = np.zeros((len(df_spb.index), 1)) # добавляем категориальные признаки 
df_moscow['Город'] = np.ones((len(df_spb.index), 1))
df = pd.concat((df_spb, df_moscow), ignore_index=True) # объединяем в единый массив

In [84]:
# разбиваем признаки на числовые, категориальные и целевой 

nums = ['Ночная температура', 'Дневная температура', 'Влажность', 'Давление', 'Сила ветра', 'Перепад температур']
cats = ['Месяц', 'Направление ветра']
targ = ['Город']

In [166]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report

In [85]:
# фиксируем random state и создаем тренировочные и тестовые выборки

RS = 2599

df_train, df_test = train_test_split(df, test_size=0.2, random_state=RS)

X_train = df_train.drop(columns='Город')
X_test = df_test.drop(columns='Город')

y_train = df_train['Город']
y_test = df_test['Город']

In [ ]:
# обработка данных с one hot encoding и стандартизация числовых признаков одинакова для всех моделей 

column_transformer = ColumnTransformer([
    ('scaling', StandardScaler(), nums),    
    ('ohe', OneHotEncoder(handle_unknown="error", drop="first"), cats)
])

## 1. Логистическая регрессия

Сначала ищем solver, потом оптимизируем доступные параметры. 

In [107]:
params = {'logreg__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

model_lr = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('logreg', LogisticRegression(random_state=RS))
])

searcher_sol = GridSearchCV(
    model_lr, 
    params, cv=5, n_jobs=-1,
    scoring='roc_auc')

In [143]:
%%capture
searcher_sol.fit(X_train, y_train)

In [144]:
searcher_sol.best_params_

{'logreg__solver': 'liblinear'}

In [145]:
y_pred = searcher_sol.predict(X_test)
p_pred = searcher_sol.predict_proba(X_test)
p_pred = p_pred[:, -1]

In [146]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.55      0.57      0.56       408
         1.0       0.59      0.57      0.58       442

    accuracy                           0.57       850
   macro avg       0.57      0.57      0.57       850
weighted avg       0.57      0.57      0.57       850



In [147]:
roc_auc_score(y_test, p_pred)

0.6026873951533875

In [148]:
alphas = np.logspace(-2, 4, 15)
params = {'logreg__penalty': ['l1', 'l2'], 'logreg__C': alphas}

model_lr = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('logreg', LogisticRegression(random_state=RS, solver='liblinear'))
])

searcher_lr = GridSearchCV(model_lr, params,
                        scoring="roc_auc", cv=5, n_jobs=-1)

In [149]:
%%capture
searcher_lr.fit(X_train, y_train)

In [150]:
y_pred = searcher_lr.predict(X_test)
p_pred = searcher_lr.predict_proba(X_test)
p_pred = p_pred[:, -1]

In [151]:
searcher_lr.best_params_

{'logreg__C': 26.826957952797247, 'logreg__penalty': 'l2'}

In [152]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.53      0.56      0.55       402
         1.0       0.59      0.56      0.57       448

    accuracy                           0.56       850
   macro avg       0.56      0.56      0.56       850
weighted avg       0.56      0.56      0.56       850



In [153]:
roc_auc_score(y_test, p_pred)

0.6028479523421972

Итого: ROC-AUC больше 0.5, но не сильно - у приемлемой модели хотелось бы видеть 0.7.

Точность и полнота совсем печальные, но они и не являются ключевой метрикой. Интересно, что Москва определяется с чуть большей точностью, чем Санкт-Петербург. 

# 2. Дерево решений 

Очень "шумный" датасет для такого алгоритма, ожидать улучшения показателей, наверное, не стоит. Но попробуем. 

In [171]:
model_dtree = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('dtree', DecisionTreeClassifier(random_state=RS))
])

params = {
    'dtree__criterion': ['gini', 'entropy'],
    'dtree__splitter' : ['best', 'random'],
    'dtree__max_features' : ["auto", "sqrt", "log2"]
}

searcher_dtree = GridSearchCV(model_dtree, params,
                        scoring="roc_auc", cv=5, n_jobs=-1)

In [173]:
%%capture
searcher_dtree.fit(X_train, y_train)

In [174]:
searcher_dtree.best_params_

{'dtree__criterion': 'entropy',
 'dtree__max_features': 'auto',
 'dtree__splitter': 'best'}

In [175]:
y_pred = searcher_dtree.predict(X_test)
p_pred = searcher_dtree.predict_proba(X_test)
p_pred = p_pred[:, -1]

In [176]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.52      0.52      0.52       427
         1.0       0.52      0.52      0.52       423

    accuracy                           0.52       850
   macro avg       0.52      0.52      0.52       850
weighted avg       0.52      0.52      0.52       850



In [177]:
roc_auc_score(y_test, p_pred)

0.520011515825956

Все очень грустно, качество модели на уровне случайного выбора. 

## 3. Случайный лес

Главный герой. Должен дать наилучшие результаты. 

In [154]:
model_rf = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('random_forest', RandomForestClassifier(random_state=RS))
])

params = {
    'random_forest__n_estimators': [100, 200, 500],
    'random_forest__max_features': ['sqrt', 'log2', 'auto']
}

searcher_rf = GridSearchCV(model_rf, params,
                        scoring="roc_auc", cv=5, n_jobs=-1)

In [178]:
%%capture
searcher_rf.fit(X_train, y_train)

In [158]:
y_pred = searcher_rf.predict(X_test)
p_pred = searcher_rf.predict_proba(X_test)
p_pred = p_pred[:, -1]

In [159]:
searcher_rf.best_params_

{'random_forest__max_features': 'sqrt', 'random_forest__n_estimators': 500}

In [160]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.65      0.62      0.63       442
         1.0       0.60      0.63      0.62       408

    accuracy                           0.62       850
   macro avg       0.62      0.62      0.62       850
weighted avg       0.63      0.62      0.62       850



In [161]:
roc_auc_score(y_test, p_pred)

0.6626582734012103

Так и есть, результаты значительно улучшились. 
Почти получилось достать до условной нижней границы приемлемого - 0.7 по ключевой метрике. 

Тут, кстати, более точно получается предсказывать не Москву, как у логистической регрессии, а Санкт-Петербург. 

В итоге можно оценить решение задачи классификации на "удовлетворительно". Если обогатить датасет информацией (например, тем же количеством осадков в милиметрах) - вполне вероятно дотянуть до "хорошо". 